# Importing Warning Help us to remove Non relevant Warning pop up During the Process 

In [164]:
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Gensim Packages For Preprocessing the Textual Data

In [165]:
import gensim
from gensim import utils, corpora, matutils, models 
from gensim.test.utils import common_texts



# Importing  User defined Modules as well as user defined

In [166]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict

# Connecting MongoDB Data Connection to Fetch Records and Update

In [167]:
from pymongo import MongoClient
client = MongoClient() # class of mongoDb
client = MongoClient('localhost', 27017) # connection port

mydb = client['New_Data_Email']  # Conneting DataBase using connection port
mydb.collection_names()

['kmean_clus',
 'Cls_Data_500',
 'Beast_kmean_Data',
 'Sample20',
 'Dump_1',
 'LDA_Database',
 'Cls_Data',
 'aff_clus',
 'Fzy_Kmean']

In [168]:
collection_name = mydb["Sample20"]
collection_name

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'New_Data_Email'), 'Sample20')

# Changing the Path of Directory where the Result of Preprocessing Get saved

In [169]:
import os
Folder_Name = str("Sep_Kmean_Lda_Genism_Output_Data")
Base_path = str("D:\\1_JD_IPYTHON\\")
def Change_dir_path(Base_path,Folder_Name):
    Change_dir = os.chdir(Base_path+Folder_Name)
Change_dir_path(Base_path,Folder_Name)
print (os.getcwd())

D:\1_JD_IPYTHON\Sep_Kmean_Lda_Genism_Output_Data


# Fetching Records from Data Base Into Pandas Dat Frame

In [170]:
Data_email = pd.DataFrame(list(collection_name.find()))
Data_email.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
Body_       20000 non-null object
Date_       20000 non-null object
Doc_N       20000 non-null int64
From_       20000 non-null object
Subject_    20000 non-null object
TO_         19156 non-null object
_id         20000 non-null object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


# Creating New Pandas Data Frame for Safer Side Copy Records In new One

In [171]:
New_Email_Df = Data_email
New_Email_Df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
Body_       20000 non-null object
Date_       20000 non-null object
Doc_N       20000 non-null int64
From_       20000 non-null object
Subject_    20000 non-null object
TO_         19156 non-null object
_id         20000 non-null object
dtypes: int64(1), object(6)
memory usage: 1.1+ MB


In [172]:
New_Email_Df.columns

Index(['Body_', 'Date_', 'Doc_N', 'From_', 'Subject_', 'TO_', '_id'], dtype='object')

# Joblib for Parallel Exceution of Data
## Importing Preprocee file which Clean the Non Relevant words
#### URL links, To ,From, Chain Email Sender and Recivers Names ,com etc

In [173]:
import Cleaning_email_body_updated_JD
from Cleaning_email_body_updated_JD import Cleaning_email_pandas
from joblib import Parallel , delayed
New_Email_Df = New_Email_Df[0:500]
Body_text = Parallel(n_jobs=3)(delayed(Cleaning_email_pandas)(x)for x in New_Email_Df["Body_"])


In [174]:
len(Body_text)

500

In [175]:
from Cleaning_email_body_updated_JD import  Gensim_preprocess
Gensim_Result = Parallel(n_jobs=2,backend='threading')(delayed(Gensim_preprocess)(text) for text in Body_text)
len(Gensim_Result),type(Gensim_Result)

(500, list)

# Below Code is removing or filter words which suggested by user

In [176]:
from Cleaning_email_body_updated_JD import  Next_Level_Word_Clean
NEW_Clean_Corpus = Parallel(n_jobs=2,backend='threading')(delayed(Next_Level_Word_Clean)(text) for text in Gensim_Result)


In [177]:
Clean_text_email = [" ".join(x)  for x in NEW_Clean_Corpus] # Rejoing the words
New_Email_Df["Doc_Number"]   = [ int(x+1) for x in range(len(New_Email_Df))] # Giving Document Seq..

C:\Users\jkdadhich\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [178]:
len(NEW_Clean_Corpus)

500

# Creating Dictionary of Corpus how many time words appears in entire text

In [179]:
Change_dir_path(Base_path,Folder_Name)
def Dic_TFIDF_Corpus(texts):
    #File_path_detail_log.Gensim_Results()
    dictionary = corpora.Dictionary(texts)
    dictionary.filter_extremes(no_below=2)
    dictionary.compactify()
    dictionary.save('dict_1.dict')
    print (dictionary)
    # Creating Verctors
    corpus = [dictionary.doc2bow(text) for text in texts]

    return corpus

Corpus_email =    Dic_TFIDF_Corpus(NEW_Clean_Corpus) 

Dictionary(4163 unique tokens: ['contractors', 'mckinsey', 'removed', 'end', 'catch']...)


# Performing the TF ~ IDF (term frequency and Inverse frequency

In [180]:
tfidf_email = models.TfidfModel(Corpus_email)
Corpus_tfidf_email = tfidf_email[Corpus_email]
Corpus_tfidf_email

# Performing LDA Topic Modeling With User Pass number of Topic

Important Factor are number of Topic, number of passes and alpa & beta however we used default right now (under research)
Dictionary as well


In [181]:
Change_dir_path(Base_path,Folder_Name)
dictionary_email = gensim.corpora.Dictionary.load("dict_1.dict")
Number_of_Topics = 10
Lda_model_user = models.LdaModel(Corpus_tfidf_email, id2word=dictionary_email, 
                      num_topics=Number_of_Topics,minimum_probability=0,
                      random_state=2,passes = 10,
                      per_word_topics=False,
                      )
Lda_model_user.save("LDA_Model.save")

# Creating Index for Document Similarity (Using Cosine In backend)

In [182]:
from gensim import corpora, models, similarities
index_email = similarities.MatrixSimilarity(Lda_model_user[Corpus_tfidf_email])
len(index_email)

500

#  Taking Out the One Dimension array from index

In [183]:
One_d_index = np.asanyarray(index_email)
One_d_index_Fetch = One_d_index[0]
One_d_index_Fetch = One_d_index_Fetch.reshape((-1,1))
One_d_index_Fetch.shape

(500, 1)

# Performing the NLTK Cosine Clustering Using Index as Vector

In [184]:
import random
Random_Value = random.randint(1,1)
import nltk
from nltk.cluster.kmeans import KMeansClusterer
NUM_CLUSTERS = Number_of_Topics
kclusterer = KMeansClusterer(NUM_CLUSTERS, 
                             distance=nltk.cluster.util.cosine_distance, 
                             repeats=25)

NLTK_clusters = kclusterer.cluster(index_email, assign_clusters=True)
from sklearn import metrics
silhouette_score = metrics.silhouette_score(index_email, NLTK_clusters, metric='cosine')
print ("Silhouette_score  :\t",round(silhouette_score*100,2))

Silhouette_score  :	 59.83


# Applying Kmean Clustering On the One D array which created above

In [185]:
from sklearn.cluster import KMeans
kmeans_email = KMeans(n_clusters=Number_of_Topics, random_state=0,max_iter=10).fit(One_d_index_Fetch)
kmeans_email.labels_
from sklearn import metrics
silhouette_score = metrics.silhouette_score(One_d_index_Fetch, kmeans_email.labels_, metric='euclidean')
print ("Silhouette_score  :\t",round(silhouette_score*100,2))

Silhouette_score  :	 59.81


# Clustering Using Topic Word Probability wrt to document

In [186]:
corpus_lda      = Lda_model_user[Corpus_tfidf_email]
lda_corpus_porb = [max(prob,key=lambda y:y[1])for prob in corpus_lda ]
lists_label     = [ x[0] for i, x in enumerate(lda_corpus_porb) ]
Max_prob        = [ round(x[1]*100,4) for i, x in enumerate(lda_corpus_porb) ]

In [187]:
from sklearn import metrics
silhouette_score = metrics.silhouette_score(lda_corpus_porb, lists_label, metric='euclidean')
print ("Silhouette_score  :\t",round(silhouette_score*100,2))

Silhouette_score  :	 82.32


# Updating the Result over the Pandas Using Topic Word Probability

In [188]:
import pandas as pd
Data_frame = New_Email_Df.copy()

Data_frame["SIM_Matirx"]  =  index_email
Data_frame["TF_IDF_LDA"]  =  [ x for x in corpus_lda]
Data_frame["Corpus"]      =  [ x for x in Corpus_email]
Data_frame["Prob_Label"]  =  lists_label
Data_frame["Max_Prob"]    =  Max_prob

Data_frame.head()

,Body_,Date_,Doc_N,From_,Subject_,TO_,_id,Doc_Number,SIM_Matirx,TF_IDF_LDA,Corpus,Prob_Label,Max_Prob
0,---------------------- forwarded by phillip m ...,"Wed, 7 Jun 2000 11:43:00 -0700 (PDT)",294217,phillip.love@enron.com,Transport,delma.salazar@enron.com,5b4cbc56f63a094ed82a0e45,1,"(1.0000001, 0.069377705, 0.09498313, 0.0854087...","[(0, 0.032012727), (1, 0.7118682), (2, 0.03201...","[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]",1,71.1868
1,"as per our discussion, can you please send a d...","Fri, 1 Jun 2001 04:39:00 -0700 (PDT)",193674,justin.rostant@enron.com,NDA request,tana.jones@enron.com,5b4cbc55f63a094ed8288586,2,"(0.069377705, 0.99999994, 0.053943243, 0.05330...","[(0, 0.015857045), (1, 0.01585025), (2, 0.8573...","[(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, ...",2,85.7317
2,"hello,\n\n an internal meeting to discuss tran...","Tue, 31 Oct 2000 04:39:00 -0800 (PST)",469031,laura.wente@enron.com,Transmission Meetings,"don.hammond@enron.com, stan.gray@enron.com, an...",5b4cbc57f63a094ed82cb923,3,"(0.09498313, 0.053943243, 0.99999994, 0.072800...","[(0, 0.015469199), (1, 0.015470081), (2, 0.015...","[(11, 1), (41, 2), (42, 1), (43, 1), (44, 1), ...",9,47.5736
3,----- forwarded by cindy derecskey/corp/enron ...,"Tue, 8 May 2001 01:16:00 -0700 (PDT)",435164,cindy.derecskey@enron.com,ADDITIONAL DOCUMENT - WSJ Documents for Ken Lay,"richard.shapiro@enron.com, steven.kean@enron.c...",5b4cbc57f63a094ed82c34d8,4,"(0.085408755, 0.05330132, 0.072800055, 0.99999...","[(0, 0.023971025), (1, 0.023974534), (2, 0.023...","[(0, 1), (87, 3), (88, 2), (89, 1), (90, 1), (...",5,78.4217
4,"on saturday, june 9, tropical storm allison le...","Mon, 11 Jun 2001 08:57:00 -0700 (PDT)",384278,office.chairman@enron.com,Houston Flood Relief Effort,all.worldwide@enron.com,5b4cbc57f63a094ed82b6e12,5,"(0.06896205, 0.2146638, 0.646569, 0.051177546,...","[(0, 0.009966641), (1, 0.009963509), (2, 0.152...","[(34, 3), (71, 1), (79, 1), (99, 1), (100, 1),...",8,76.7558


In [189]:
Data_frame.columns

Index(['Body_', 'Date_', 'Doc_N', 'From_', 'Subject_', 'TO_', '_id',
       'Doc_Number', 'SIM_Matirx', 'TF_IDF_LDA', 'Corpus', 'Prob_Label',
       'Max_Prob'],
      dtype='object')

# Shorting the Data Frame for filter out the Next level Processing

In [190]:
Data_frame['Rank_Prob'] = Data_frame.groupby('Prob_Label')['Max_Prob'].rank(ascending=False)
Data_frame.sort_values(by=['Prob_Label',"Rank_Prob"], ascending=[True,True],inplace=True)

# Calulating the Cosine Distance (Vector's of Documents)

In [191]:
from scipy.spatial.distance import cosine,euclidean

def Cos_SIM_Funct(X,Y):  
    #X = [x[1] for x in X]
    #Y = [x[1] for x in Y]
    return round((1 - cosine(X,Y))*100,4)



In [192]:
def Cosine_mean_Sim(Cls_value):
    Temp_df = Data_frame.loc[Data_frame.Prob_Label == int(Cls_value)]
    Select_RanK_1 = Temp_df["SIM_Matirx"].values[0]
   # Select_RanK_1 = Temp_df["TF_IDF_LDA"].values[0]
    Value_cos_dis = []
    print ("~~@~~"*10)
    for x in Temp_df.SIM_Matirx:
        Value_cos_dis.append(Cos_SIM_Funct(Select_RanK_1,x))
         
    Temp_df["Cos_Simlarity"]  = Value_cos_dis
    print (Value_cos_dis)
    return Temp_df
    
        
    

# Updating the Records Into Pandas Data Frame and also caluating the Distance cosine wrt of Hight Prob. of Document in that cluster

In [193]:
blank_df = pd.DataFrame(columns=Data_frame.columns) 
for cls_ in range(Number_of_Topics):
    blank_df = pd.concat([blank_df,Cosine_mean_Sim(Cls_value=cls_)])
    

~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~
[100.0, 100.0, 100.0, 99.9975, 99.9713, 99.957, 99.9423, 99.9118, 99.9013, 99.8749, 97.5623, 99.7218, 98.9785, 92.6798, 98.5986, 92.6578, 85.125, 84.1873, 97.6887, 97.6763, 83.4854, 81.4253, 74.1386, 78.833, 75.5063, 77.9097, 82.8635, 71.5847, 65.8181, 48.7248, 48.7248, 48.7248, 48.7248, 48.7248]
~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~
[100.0, 99.9663, 99.9663, 99.9663, 99.9663, 99.9663, 99.9663, 99.9663, 99.9614, 99.9477, 99.9322, 99.9264, 99.9156, 99.8714, 99.8695, 99.8659, 99.8636, 99.8222, 99.7934, 99.7184, 97.6267, 96.9065, 99.5529, 99.5357, 99.4848, 99.3992, 99.2917, 92.2781, 99.0896, 95.1922, 93.921, 98.2397, 92.0996, 89.2603, 98.0553, 85.9779, 89.8663, 84.2521, 84.775, 79.5128, 96.3117, 79.2054, 71.9502, 69.6009, 68.0402, 82.2691, 73.0229, 65.2174, 61.8918]
~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~


C:\Users\jkdadhich\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


[100.0, 99.9996, 99.9986, 99.9975, 99.9971, 99.9971, 99.9966, 99.9955, 99.9879, 99.9858, 99.982, 99.9674, 99.9656, 99.965, 99.9648, 99.9626, 99.5301, 99.9437, 99.9368, 99.9364, 99.9315, 99.9289, 99.9273, 99.9252, 99.9105, 99.9001, 99.8808, 99.7913, 99.3287, 97.9373, 99.6593, 96.5867, 95.3574, 94.5824, 94.5808, 96.1105, 99.1447, 98.4613, 91.0109, 97.9024, 88.0898, 89.7803, 87.4399, 85.9798, 84.6646, 86.2439, 83.3053, 83.5946, 83.3008, 97.1171, 80.0081, 85.482, 87.2705, 77.4505, 80.1391, 90.6443, 82.9652, 79.8687, 77.0051, 76.0151, 66.5391, 70.0937]
~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~
[100.0, 99.9916, 99.9914, 99.989, 99.8791, 96.7454, 99.7618, 99.5369, 99.4806, 99.4176, 99.2009, 90.8573, 98.8754, 98.8754, 83.715, 97.6291, 83.7441, 95.7412, 95.7411, 84.7142, 73.6146, 83.4502, 75.2837, 71.6947, 71.24, 74.3033, 71.5879, 69.8973, 73.5218, 73.0976, 60.8623, 57.3641]
~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~~~@~~
[100.0, 99.9859, 99.9391, 99.9371, 99.902, 99.7741, 99.7127, 

In [194]:
blank_df.head()

,Body_,Corpus,Cos_Simlarity,Date_,Doc_N,Doc_Number,From_,Max_Prob,Prob_Label,Rank_Prob,SIM_Matirx,Subject_,TF_IDF_LDA,TO_,_id
440,"dave,\n\nwe had a meeting today with mark frev...","[(12, 1), (53, 1), (64, 2), (208, 1), (219, 1)...",100.0000,"Mon, 17 Jul 2000 11:25:00 -0700 (PDT)",263373,441,kimberly.hillis@enron.com,84.2785,0,1.0,"(0.0723559, 0.04214374, 0.057439364, 0.0560173...",Alberta PPA,"[(0, 0.8427851), (1, 0.017466994), (2, 0.01746...",david.delainey@enron.com,5b4cbc56f63a094ed82995c9
276,i will be out on th & fri because my wife is h...,"[(42, 1), (113, 1), (162, 1), (169, 1), (177, ...",100.0000,"Wed, 24 Jan 2001 08:32:00 -0800 (PST)",120775,277,rob.gay@enron.com,84.2707,0,2.0,"(0.07236882, 0.04215669, 0.05746351, 0.0560301...",Wife's surgery and Bersani,"[(0, 0.8427131), (1, 0.017473105), (2, 0.01747...",peter.weidler@enron.com,5b4cbc54f63a094ed82768c3
418,can you kill this deal or zero out the volumes...,"[(38, 3), (84, 1), (156, 2), (192, 1), (287, 2...",100.0000,"Wed, 25 Apr 2001 08:13:00 -0700 (PDT)",468500,419,rhonda.denton@enron.com,84.2613,0,3.0,"(0.07239429, 0.0421756, 0.05748879, 0.05604868...",Re: 581368,"[(0, 0.84261274), (1, 0.017488237), (2, 0.0174...",kate.symes@enron.com,5b4cbc57f63a094ed82cb710
319,this deal was not put in right. the primary t...,"[(0, 1), (38, 1), (76, 1), (79, 1), (82, 1), (...",99.9975,"Wed, 13 Sep 2000 06:37:00 -0700 (PDT)",108318,320,julie.meyers@enron.com,83.4331,0,4.0,"(0.074123114, 0.043649085, 0.0595716, 0.057638...",Re: Meter 6879 - Sept. 00,"[(0, 0.8343554), (1, 0.01839862), (2, 0.018399...",daren.farmer@enron.com,5b4cbc54f63a094ed827381a
167,again mark i am flattered that you include me ...,"[(160, 1), (169, 1), (256, 2), (288, 3), (641,...",99.9713,"Fri, 28 Apr 2000 03:45:00 -0700 (PDT)",410864,168,susan.scott@enron.com,81.4350,0,5.0,"(0.07847097, 0.047362793, 0.06464015, 0.061656...",Re: 5 day loss limit violation (4/20 -4/27/2000),"[(0, 0.8143492), (1, 0.020624613), (2, 0.02062...",mark.fondren@enron.com,5b4cbc57f63a094ed82bd5ec


# User Defined function to evaluate the STD of Vector's

In [195]:
def return_pos(mean_,limit_pos):
    #Index_STD = []
    Pos_1 = (list(limit_pos.values())[0])
    Pos_2 = (list(limit_pos.values())[1])
    Pos_3 = (list(limit_pos.values())[2])
    Pos_4 = (list(limit_pos.values())[3])
    Pos_5 = (list(limit_pos.values())[4])
    Pos_6 = (list(limit_pos.values())[5])
    Pos_7 = (list(limit_pos.values())[6])
    Pos_8 = (list(limit_pos.values())[7])
    Pos_9 = (list(limit_pos.values())[8])
    Pos_10 = (list(limit_pos.values())[9])
 
    if Pos_1 <= mean_ <= Pos_2:
        Index_STD =(list(limit_pos.keys())[0])   
    elif Pos_2 <= mean_ <= Pos_3:
        Index_STD = (list(limit_pos.keys())[1])
    elif Pos_3 <= mean_ <= Pos_4:
        Index_STD =(list(limit_pos.keys())[2])
    elif Pos_4 <= mean_ <= Pos_5:
        Index_STD =(list(limit_pos.keys())[3])
    elif Pos_5 <= mean_ <= Pos_6:
        Index_STD =(list(limit_pos.keys())[4])
    elif Pos_6 <= mean_ <= Pos_7:
        Index_STD =(list(limit_pos.keys())[5])
    elif Pos_7 <= mean_ <= Pos_8:
        Index_STD =(list(limit_pos.keys())[6])
    elif Pos_8 <= mean_ <= Pos_9:
        Index_STD =(list(limit_pos.keys())[7])
    elif Pos_9 <= mean_ <= Pos_10:
        Index_STD =(list(limit_pos.keys())[8])
    else:
        Index_STD =(2.50)        
    return Index_STD

def return_neg(mean_,limit_pos):
    #Index_STD = []
    Pos_1 = (list(limit_pos.values())[0])
    Pos_2 = (list(limit_pos.values())[1])
    Pos_3 = (list(limit_pos.values())[2])
    Pos_4 = (list(limit_pos.values())[3])
    Pos_5 = (list(limit_pos.values())[4])
    Pos_6 = (list(limit_pos.values())[5])
    Pos_7 = (list(limit_pos.values())[6])
    Pos_8 = (list(limit_pos.values())[7])
    Pos_9 = (list(limit_pos.values())[8])
    Pos_10 = (list(limit_pos.values())[9])
    if Pos_1 >= mean_ >= Pos_2:
        Index_STD =(list(limit_pos.keys())[0])
    elif Pos_2 >= mean_ >= Pos_3:
        Index_STD = (list(limit_pos.keys())[1])
    elif Pos_3 >= mean_ >= Pos_4:
        Index_STD =(list(limit_pos.keys())[2])
    elif Pos_4 >= mean_ >= Pos_5:
        Index_STD =(list(limit_pos.keys())[3])
    elif Pos_5 >= mean_ >= Pos_6:
        Index_STD =(list(limit_pos.keys())[4])
    elif Pos_6 >= mean_ >= Pos_7:
        Index_STD =(list(limit_pos.keys())[5])
    elif Pos_7 >= mean_ >= Pos_8:
        Index_STD =(list(limit_pos.keys())[6])
    elif Pos_8 >= mean_ >= Pos_9:
        Index_STD =(list(limit_pos.keys())[7])
    elif Pos_9 >= mean_ >= Pos_10:
        Index_STD =(list(limit_pos.keys())[8])
    else:
        Index_STD =(2.50)
    return Index_STD

# Mapping the STD and Updating the Database with Pandas Df

In [196]:
import numpy as np
def STD_Sampling_Data_JD_Sep_5(Df,dbname):
    MAD = Df.Cos_Simlarity.std(ddof=0)
    Place_Value = np.where(Df.Cos_Simlarity == float(100.0000))
    print (Place_Value)
    Best_Dist   = Df.Cos_Simlarity.iloc[Place_Value[0]]
    if len(Best_Dist)==1:
        Best_Dist = float(Best_Dist)
    else:
        Best_Dist = 0
    print ("Best Distance",Best_Dist)
    
    Threhold_List = [0,0.25,0.50,0.75,1.0,1.25,1.50,1.75,2.0,2.25]
    Positive_Limit = [round(Best_Dist + (thresh * MAD),6) for thresh in Threhold_List]
    
    Dict_Positive_limit = dict(zip(Threhold_List, Positive_Limit))
    
    Negative_Limit = [round(Best_Dist - (thresh * MAD),6) for thresh in Threhold_List]
    
    Dict_Negative_Limit = dict(zip(Threhold_List, Negative_Limit))
    
    Df["Pos"] =Df.Cos_Simlarity.apply(lambda x: return_pos(x,Dict_Positive_limit))
    Df["Neg"] =Df.Cos_Simlarity.apply(lambda x: return_neg(x,Dict_Negative_Limit))
    
    
    
    List_Get = [x if x <=y  and x <= 2.50 else y for x,y in zip(Df["Pos"],Df["Neg"])]
    Df["Pos"]
    Df["Neg"]
    Df["Response"] = List_Get
#    Df.MATRIX = Df["MATRIX"].astype(str)
    Df.SIM_Matirx = Df["SIM_Matirx"].astype(str)
    Df.TF_IDF_LDA = Df["TF_IDF_LDA"].astype(str)
    #Df.Max_Prob = Df["Max_Prob"].astype(str)
    #Df.Cos_Simlarity = Df["Cos_Simlarity"].astype(str)
    
    
    print (Df.info())
    
    dbname = str(dbname)
    collection_name = mydb[dbname]
    collection_name.insert_many(Df.to_dict('records'))
    

# Calling the above functions to perform task

In [197]:
from pymongo import MongoClient # MongoDB
client = MongoClient() # class of mongoDb
client = MongoClient('localhost', 27017) # connection port
mydb = client['New_Data_Email']
dbname = "LDA_Database"

def Clean_DB_First_JD(dbname):
    mydb = client['New_Data_Email']
    #print (mydb.collection_names())
    
    collection_name = mydb[dbname]
    print (collection_name.name)
    
    #print (collection_name.count())
    collection_name.remove()
    
Clean_DB_First_JD(dbname)

for j in range(Number_of_Topics):
    print ("Cluster Number is Processed :\t",j)
    Demo = blank_df.loc[blank_df.Prob_Label == j]
    STD_Sampling_Data_JD_Sep_5(Demo,dbname)
    

LDA_Database
Cluster Number is Processed :	 0
(array([0, 1, 2], dtype=int64),)
Best Distance 0


C:\Users\jkdadhich\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jkdadhich\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jkdadhich\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 440 to 491
Data columns (total 18 columns):
Body_            34 non-null object
Corpus           34 non-null object
Cos_Simlarity    34 non-null float64
Date_            34 non-null object
Doc_N            34 non-null object
Doc_Number       34 non-null object
From_            34 non-null object
Max_Prob         34 non-null float64
Prob_Label       34 non-null object
Rank_Prob        34 non-null float64
SIM_Matirx       34 non-null object
Subject_         34 non-null object
TF_IDF_LDA       34 non-null object
TO_              34 non-null object
_id              34 non-null object
Pos              34 non-null float64
Neg              34 non-null float64
Response         34 non-null float64
dtypes: float64(6), object(12)
memory usage: 5.0+ KB
None
Cluster Number is Processed :	 1
(array([0], dtype=int64),)
Best Distance 100.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 18 to 116
Data columns (total 18 columns

In [198]:
len(blank_df)

500

In [199]:
Data_email_Result = pd.DataFrame(list(mydb.LDA_Database.find()))
len(Data_email_Result)

500

In [200]:
Data_email_Result.columns

Index(['Body_', 'Corpus', 'Cos_Simlarity', 'Date_', 'Doc_N', 'Doc_Number',
       'From_', 'Max_Prob', 'Neg', 'Pos', 'Prob_Label', 'Rank_Prob',
       'Response', 'SIM_Matirx', 'Subject_', 'TF_IDF_LDA', 'TO_', '_id'],
      dtype='object')

In [201]:
Data_email_Result.head()

,Body_,Corpus,Cos_Simlarity,Date_,Doc_N,Doc_Number,From_,Max_Prob,Neg,Pos,Prob_Label,Rank_Prob,Response,SIM_Matirx,Subject_,TF_IDF_LDA,TO_,_id
0,"dave,\n\nwe had a meeting today with mark frev...","[[12, 1], [53, 1], [64, 2], [208, 1], [219, 1]...",100.0000,"Mon, 17 Jul 2000 11:25:00 -0700 (PDT)",263373,441,kimberly.hillis@enron.com,84.2785,2.5,2.5,0,1.0,2.5,"(0.0723559, 0.04214374, 0.057439364, 0.0560173...",Alberta PPA,"[(0, 0.8427851), (1, 0.017466994), (2, 0.01746...",david.delainey@enron.com,5b4cbc56f63a094ed82995c9
1,i will be out on th & fri because my wife is h...,"[[42, 1], [113, 1], [162, 1], [169, 1], [177, ...",100.0000,"Wed, 24 Jan 2001 08:32:00 -0800 (PST)",120775,277,rob.gay@enron.com,84.2707,2.5,2.5,0,2.0,2.5,"(0.07236882, 0.04215669, 0.05746351, 0.0560301...",Wife's surgery and Bersani,"[(0, 0.8427131), (1, 0.017473105), (2, 0.01747...",peter.weidler@enron.com,5b4cbc54f63a094ed82768c3
2,can you kill this deal or zero out the volumes...,"[[38, 3], [84, 1], [156, 2], [192, 1], [287, 2...",100.0000,"Wed, 25 Apr 2001 08:13:00 -0700 (PDT)",468500,419,rhonda.denton@enron.com,84.2613,2.5,2.5,0,3.0,2.5,"(0.07239429, 0.0421756, 0.05748879, 0.05604868...",Re: 581368,"[(0, 0.84261274), (1, 0.017488237), (2, 0.0174...",kate.symes@enron.com,5b4cbc57f63a094ed82cb710
3,this deal was not put in right. the primary t...,"[[0, 1], [38, 1], [76, 1], [79, 1], [82, 1], [...",99.9975,"Wed, 13 Sep 2000 06:37:00 -0700 (PDT)",108318,320,julie.meyers@enron.com,83.4331,2.5,2.5,0,4.0,2.5,"(0.074123114, 0.043649085, 0.0595716, 0.057638...",Re: Meter 6879 - Sept. 00,"[(0, 0.8343554), (1, 0.01839862), (2, 0.018399...",daren.farmer@enron.com,5b4cbc54f63a094ed827381a
4,again mark i am flattered that you include me ...,"[[160, 1], [169, 1], [256, 2], [288, 3], [641,...",99.9713,"Fri, 28 Apr 2000 03:45:00 -0700 (PDT)",410864,168,susan.scott@enron.com,81.4350,2.5,2.5,0,5.0,2.5,"(0.07847097, 0.047362793, 0.06464015, 0.061656...",Re: 5 day loss limit violation (4/20 -4/27/2000),"[(0, 0.8143492), (1, 0.020624613), (2, 0.02062...",mark.fondren@enron.com,5b4cbc57f63a094ed82bd5ec


In [202]:
Data_email_Result.to_excel("Sample_.xlsx")